In [1]:
import csv
import math
import string
import itertools
from io import open
from conllu import parse_incr
import torch
import torch.nn as nn
from nltk.corpus import wordnet as wn
import numpy as np

In [2]:
# parse the WSD dataset first
# and retrieve all sentences from the EUD

'''
Copyright@
White, A. S., D. Reisinger, K. Sakaguchi, T. Vieira, S. Zhang, R. Rudinger, K. Rawlins, & B. Van Durme. 2016. 
[Universal decompositional semantics on universal dependencies]
(http://aswhite.net/media/papers/white_universal_2016.pdf). 
To appear in *Proceedings of the Conference on Empirical Methods in Natural Language Processing 2016*.
'''

def parse_wsd_data():

    # parse the EUD-EWT conllu files and retrieve the sentences
    # remove all punctuation?
    train_file = open("data/UD_English-EWT/en_ewt-ud-train.conllu", "r", encoding="utf-8")
    train_data = list(parse_incr(train_file))
    # train_data = [[''.join(c for c in word.get('lemma') if c not in string.punctuation) for word in token_list] for token_list in train_data]
    # train_data = [[word for word in s if word] for s in train_data]
    print('Parsed {} training data from UD_English-EWT/en_ewt-ud-train.conllu.'.format(len(train_data)))

    test_file = open("data/UD_English-EWT/en_ewt-ud-test.conllu", "r", encoding="utf-8")
    test_data = list(parse_incr(test_file))
    # test_data = [[''.join(c for c in word.get('lemma') if c not in string.punctuation) for word in token_list] for token_list in test_data]
    # test_data = [[word for word in s if word] for s in test_data]
    print('Parsed {} testing data from UD_English-EWT/en_ewt-ud-test.conllu.'.format(len(test_data)))

    dev_file = open("data/UD_English-EWT/en_ewt-ud-dev.conllu", "r", encoding="utf-8")
    dev_data = list(parse_incr(dev_file))
    # dev_data = [[''.join(c for c in word.get('lemma') if c not in string.punctuation) for word in token_list] for token_list in dev_data]
    # dev_data = [[word for word in s if word] for s in dev_data]
    print('Parsed {} dev data from UD_English-EWT/en_ewt-ud-dev.conllu.'.format(len(dev_data)))

    # parse the WSD dataset
    wsd_data = []

    # read in tsv by White et. al., 2016
    with open('data/wsd/wsd_eng_ud1.2_10262016.tsv', mode = 'r') as wsd_file:

        tsv_reader = csv.DictReader(wsd_file, delimiter = '\t')      

        # store the data: ordered dict row
        for row in tsv_reader:                                
                            
            # each data vector
            wsd_data.append(row)

        # make sure all data are parsed
        print('Parsed {} word sense data from White et. al., 2016.'.format(len(wsd_data)))

    return wsd_data, train_data, test_data, dev_data

In [3]:
# get the raw wsd data
wsd_data, train_data, test_data, dev_data = parse_wsd_data()

Parsed 12543 training data from UD_English-EWT/en_ewt-ud-train.conllu.
Parsed 2077 testing data from UD_English-EWT/en_ewt-ud-test.conllu.
Parsed 2002 dev data from UD_English-EWT/en_ewt-ud-dev.conllu.
Parsed 439312 word sense data from White et. al., 2016.


In [4]:
'''
return: 
all senses for each word 
all definitions for each word
all supersenses
from the EUD for train, test, and dev dataset
index provided by WSD dataset by White et. al.
'''
# get all the senses and definitions for each word from WSD dataset
# order of senses and definitions are in order
def get_all_senses_and_definitions(wsd_data, train_data, test_data, dev_data):

    # all senses for each word in train and dev
    # supersense is shared 
    all_senses = {}
    all_definitions = {}
    all_supersenses = {}

    # all senses for each word in test
    all_test_senses = {}
    all_test_definitions = {}
    
    # only get the senses for train and dev set
    for i in range(len(wsd_data)):
        
        # get the original sentence from EUD
        sentence_id = wsd_data[i].get('Sentence.ID')
        
        # get the definitions for the target word from EUD
        definition = wsd_data[i].get('Sense.Definition').split(' ')
        
        # the index in EUD is 1-based!!!
        sentence_number = int(sentence_id.split(' ')[-1]) - 1
        word_index = int(wsd_data[i].get('Arg.Token')) - 1
        
        word_lemma = wsd_data[i].get('Arg.Lemma')
        word_sense = wsd_data[i].get('Synset')
        response = wsd_data[i].get('Sense.Response')
        
        # add a under score to avoid name conflict with pytorch build-in attributes
        # get the original word
        # in case of errors in the dataset
        # correct it to the original word the annotator saw
        # add a under score to avoid name conflict with pytorch build-in attributes
        old = '____' + word_lemma
        if wsd_data[i].get('Split') == 'train':
            sentence = train_data[sentence_number]
            word_lemma = '____' + [word.get('lemma') for word in sentence][word_index]
        elif wsd_data[i].get('Split') == 'test':
            sentence = test_data[sentence_number]
            word_lemma = '____' + [word.get('lemma') for word in sentence][word_index]
        else:
            sentence = dev_data[sentence_number]
            word_lemma = '____' + [word.get('lemma') for word in sentence][word_index]
        
        # index error in UD: some sentences start with '<<'
        # have wrong index
        if [word.get('lemma') for word in sentence][0] == '<<' and [word.get('lemma') for word in sentence][-1] != '>>':
            if '____' + [word.get('lemma') for word in sentence][word_index] != old:
                word_lemma = old
                
        # senses for train and dev
        # preserve unknown words
        if wsd_data[i].get('Split') != 'test':
        
            # supersense-> (word_lemma, word_sense) dictionary
            super_s = wn.synset(word_sense).lexname().replace('.', '_')
            if all_supersenses.get(super_s, 'not_exist') != 'not_exist':
                all_supersenses[super_s].add((word_lemma, word_sense))
            else:
                all_supersenses[super_s] = {(word_lemma, word_sense)}

            # if the word already exits: add the new sense to the list
            # else: creata a new list for the word
            if all_senses.get(word_lemma, 'not_exist') != 'not_exist':
                if word_sense not in all_senses[word_lemma]:
                    all_senses[word_lemma].append(word_sense)
            else:
                all_senses[word_lemma] = []
                all_senses[word_lemma].append(word_sense)            
            
            if all_definitions.get(word_lemma,'not_exist') != 'not_exist':
                if definition not in all_definitions[word_lemma]: 
                    all_definitions[word_lemma].append(definition)
            else:
                all_definitions[word_lemma] = []
                all_definitions[word_lemma].append(definition)
                
        else:

            # all the senses and definitions for test words
            if all_test_senses.get(word_lemma, 'not_exist') != 'not_exist':
                if word_sense not in all_test_senses[word_lemma]:
                    all_test_senses[word_lemma].append(word_sense)
            else:
                all_test_senses[word_lemma] = []
                all_test_senses[word_lemma].append(word_sense)            
            
            if all_test_definitions.get(word_lemma,'not_exist') != 'not_exist':
                if definition not in all_test_definitions[word_lemma]: 
                    all_test_definitions[word_lemma].append(definition)
            else:
                all_test_definitions[word_lemma] = []
                all_test_definitions[word_lemma].append(definition)            
        
    # UD error in 'gam' and 'game'
    all_test_senses['____game'] = all_test_senses['____game'][:-1]
    all_test_definitions['____game'] = all_test_definitions['____game'][:-1]
    
    # print(all_senses['____game'])
    # print(len(all_definitions['____game']))
    # print(len(all_test_definitions['____game']))
    # print(all_test_senses['____game'])
    
    return all_senses, all_definitions, all_supersenses, all_test_senses, all_test_definitions

In [5]:
# get all the senses and definitions
all_senses, all_definitions, all_supersenses, all_test_senses, all_test_definitions = get_all_senses_and_definitions(wsd_data, train_data, test_data, dev_data)


In [6]:
# read the train, dev, test datasets from processed files
# check the 'data_loader.ipynb' for details
def read_file():
    
    train_X = []
    train_X_num = 0
    train_Y = []
    train_Y_num = 0
    test_X = []
    test_X_num = 0
    test_Y = []
    test_Y_num = 0
    dev_X = []
    dev_X_num = 0
    dev_Y = []
    dev_Y_num = 0
    
    train_word_idx = []
    test_word_idx = []
    dev_word_idx = []
    
    # read in csv
    with open('data/train_X.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file, delimiter = '\t')

        # store the data
        for row in csv_reader:

            train_X.append(row)
            train_X_num += 1

        # make sure all data are parsed
        print(f'Parsed {train_X_num} data points for train_X.')

    with open('data/train_Y.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            train_Y.append(row)
            train_Y_num += 1

        # make sure all data are parsed
        print(f'Parsed {train_Y_num} data points for train_Y.')
        
    with open('data/train_word_idx.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            train_word_idx = (row)

        # make sure all data are parsed
        print(f'Parsed {len(train_word_idx)} data points for train_word_idx.')

    with open('data/dev_X.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file, delimiter = '\t')

        # store the data
        for row in csv_reader:

            dev_X.append(row)
            dev_X_num += 1

        # make sure all data are parsed
        print(f'Parsed {dev_X_num} data points for dev_X.')

    with open('data/dev_Y.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            dev_Y.append(row)
            dev_Y_num += 1

        # make sure all data are parsed
        print(f'Parsed {dev_Y_num} data points for dev_Y.')
        
    with open('data/dev_word_idx.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            dev_word_idx = (row)

        # make sure all data are parsed
        print(f'Parsed {len(dev_word_idx)} data points for dev_word_idx.')
        
    with open('data/test_X.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file, delimiter = '\t')

        # store the data
        for row in csv_reader:

            test_X.append(row)
            test_X_num += 1

        # make sure all data are parsed
        print(f'Parsed {test_X_num} data points for test_X.')

    with open('data/test_Y.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            test_Y.append(row)
            test_Y_num += 1

        # make sure all data are parsed
        print(f'Parsed {test_Y_num} data points for test_Y.')
        
    with open('data/test_word_idx.tsv', mode = 'r') as data_file:
        
        csv_reader = csv.reader(data_file)

        # store the data
        for row in csv_reader:

            row = list(map(int, row))
            test_word_idx = (row)

        # make sure all data are parsed
        print(f'Parsed {len(test_word_idx)} data points for test_word_idx.')    
        
    return train_X, train_Y, test_X, test_Y, dev_X, dev_Y, train_word_idx, test_word_idx, dev_word_idx

In [7]:
# get all the structured data
train_X, train_Y, test_X, test_Y, dev_X, dev_Y, train_word_idx, test_word_idx, dev_word_idx = read_file()

# print('num of training examples: {}'.format(len(train_X)))
# print('num of dev examples: {}'.format(len(dev_X)))
# print('num of testing examples: {}'.format(len(test_X)))

# test on small subset
train_X = train_X[:10000]
train_Y = train_Y[:10000]
train_word_idx = train_word_idx[:10000]


# all vocab in the 10000 sentences
vocab = set()
for index, sen in enumerate(train_X):
    word = sen[train_word_idx[index]]
    if word not in vocab and ('____' + word) in all_senses.keys():
        vocab.add(word)
print('number of known words: {}'.format(len(vocab)))
print('number of train data: {}'.format(len(train_X)))

# filter the dev set
new_dev_X = []
new_dev_Y = []
new_dev_idx = []
for index, sen in enumerate(dev_X):
    word = sen[dev_word_idx[index]]
    if word in vocab and ('____' + word) in all_senses.keys():
        new_dev_idx.append(dev_word_idx[index])
        new_dev_X.append(sen)
        new_dev_Y.append(dev_Y[index])
new_dev_X = new_dev_X[:2000]
new_dev_Y = new_dev_Y[:2000]
new_dev_idx = new_dev_idx[:2000]

print('number of dev data: {}'.format(len(new_dev_X)))

# filter the test set
new_test_idx = []
new_test_X = []
new_test_Y = []
known = 0
unknown = 0
for index, sen in enumerate(test_X):
    word = sen[test_word_idx[index]]
    if word in vocab and ('____' + word) in all_senses.keys():
        new_test_idx.append(test_word_idx[index])
        new_test_X.append(sen)
        new_test_Y.append(test_Y[index])
        known += 1
    elif word not in all_senses.keys():
        new_test_idx.append(test_word_idx[index])
        new_test_X.append(sen)
        new_test_Y.append(test_Y[index])        
        unknown += 1
print('number of test data (known words): {}'.format(known))
print('number of test data (unknown words): {}'.format(unknown))


Parsed 67619 data points for train_X.
Parsed 67619 data points for train_Y.
Parsed 67619 data points for train_word_idx.
Parsed 7332 data points for dev_X.
Parsed 7332 data points for dev_Y.
Parsed 7332 data points for dev_word_idx.
Parsed 7118 data points for test_X.
Parsed 7118 data points for test_Y.
Parsed 7118 data points for test_word_idx.
number of known words: 1873
number of train data: 10000
number of dev data: 2000
number of test data (known words): 5412
number of test data (unknown words): 1706


In [8]:
from model import *
from trainer import *

from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Device: cpu
0


In [9]:
# load trained .pth file later
epochs = 0

# for demo only
trainer = Trainer(epochs = epochs, elmo_class = elmo, all_senses = all_senses, all_supersenses = all_supersenses)


In [10]:
# show the model
# small vocab
train_losses, dev_losses, dev_rs = trainer.train(train_X, train_Y, train_word_idx, new_dev_X, new_dev_Y, new_dev_idx)


#############   Model Parameters   ##############
layers.word_sense.0.weight torch.Size([512, 512])
layers.word_sense.0.bias torch.Size([512])
layers.word_sense.2.weight torch.Size([300, 512])
layers.word_sense.2.bias torch.Size([300])
dimension_reduction_MLP.weight torch.Size([256, 3072])
dimension_reduction_MLP.bias torch.Size([256])
wsd_lstm.weight_ih_l0 torch.Size([1024, 256])
wsd_lstm.weight_hh_l0 torch.Size([1024, 256])
wsd_lstm.bias_ih_l0 torch.Size([1024])
wsd_lstm.bias_hh_l0 torch.Size([1024])
wsd_lstm.weight_ih_l0_reverse torch.Size([1024, 256])
wsd_lstm.weight_hh_l0_reverse torch.Size([1024, 256])
wsd_lstm.bias_ih_l0_reverse torch.Size([1024])
wsd_lstm.bias_hh_l0_reverse torch.Size([1024])
wsd_lstm.weight_ih_l1 torch.Size([1024, 512])
wsd_lstm.weight_hh_l1 torch.Size([1024, 256])
wsd_lstm.bias_ih_l1 torch.Size([1024])
wsd_lstm.bias_hh_l1 torch.Size([1024])
wsd_lstm.weight_ih_l1_reverse torch.Size([1024, 512])
wsd_lstm.weight_hh_l1_reverse torch.Size([1024, 256])
wsd_lstm.bia

definition_embeddings.____faith torch.Size([300, 4])
definition_embeddings.____fall torch.Size([300, 12])
definition_embeddings.____fallacy torch.Size([300, 1])
definition_embeddings.____fallibility torch.Size([300, 1])
definition_embeddings.____fallout torch.Size([300, 2])
definition_embeddings.____falls torch.Size([300, 12])
definition_embeddings.____fame torch.Size([300, 2])
definition_embeddings.____family torch.Size([300, 8])
definition_embeddings.____fan torch.Size([300, 3])
definition_embeddings.____fancy torch.Size([300, 3])
definition_embeddings.____fantasy torch.Size([300, 3])
definition_embeddings.____faq torch.Size([300, 1])
definition_embeddings.____far torch.Size([300, 1])
definition_embeddings.____farce torch.Size([300, 2])
definition_embeddings.____fare torch.Size([300, 4])
definition_embeddings.____farm torch.Size([300, 1])
definition_embeddings.____farmer torch.Size([300, 3])
definition_embeddings.____farrier torch.Size([300, 1])
definition_embeddings.____fashion torc

definition_embeddings.____ok torch.Size([300, 2])
definition_embeddings.____old torch.Size([300, 1])
definition_embeddings.____omelet torch.Size([300, 1])
definition_embeddings.____omission torch.Size([300, 4])
definition_embeddings.____one torch.Size([300, 2])
definition_embeddings.____open torch.Size([300, 4])
definition_embeddings.____opening torch.Size([300, 13])
definition_embeddings.____opera torch.Size([300, 3])
definition_embeddings.____operation torch.Size([300, 11])
definition_embeddings.____operative torch.Size([300, 2])
definition_embeddings.____operator torch.Size([300, 5])
definition_embeddings.____opinion torch.Size([300, 6])
definition_embeddings.____opossum torch.Size([300, 2])
definition_embeddings.____opportunity torch.Size([300, 1])
definition_embeddings.____opposition torch.Size([300, 8])
definition_embeddings.____option torch.Size([300, 3])
definition_embeddings.____orange torch.Size([300, 5])
definition_embeddings.____orbit torch.Size([300, 5])
definition_embeddi

definition_embeddings.____tanker torch.Size([300, 2])
definition_embeddings.____tape torch.Size([300, 5])
definition_embeddings.____target torch.Size([300, 5])
definition_embeddings.____task torch.Size([300, 2])
definition_embeddings.____taste torch.Size([300, 7])
definition_embeddings.____tasting torch.Size([300, 3])
definition_embeddings.____tat torch.Size([300, 2])
definition_embeddings.____tattoo torch.Size([300, 3])
definition_embeddings.____tau torch.Size([300, 1])
definition_embeddings.____tavern torch.Size([300, 1])
definition_embeddings.____tax torch.Size([300, 1])
definition_embeddings.____tc torch.Size([300, 2])
definition_embeddings.____tea torch.Size([300, 5])
definition_embeddings.____teacher torch.Size([300, 2])
definition_embeddings.____teaching torch.Size([300, 3])
definition_embeddings.____team torch.Size([300, 2])
definition_embeddings.____technician torch.Size([300, 2])
definition_embeddings.____technique torch.Size([300, 2])
definition_embeddings.____technology tor

In [11]:
# load the trained parameters for testing
# 10000 examples
# 22 iterations
trainer._model.load_state_dict(torch.load('22_word_sense_model_.pth', map_location='cpu'))

# cut smaller test set


In [12]:
# test the model
cos = nn.CosineSimilarity(dim = 1, eps = 1e-6).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
correct_count = 0
known_test_size = 0
unknown_test_size = 0
unknown_correct_count = 0

# overall accuracy
for test_idx, test_sen in enumerate(new_test_X):
    
    test_lemma = '____' + test_sen[new_test_idx[test_idx]]
    test_emb = trainer._model.forward(test_sen, new_test_idx[test_idx]).view(1, -1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    all_similarity = []
    
    # if it is a new word
    # only test on the supersense
    if all_senses.get(test_lemma, 'e') == 'e':
        
        unknown_test_size += 1
        test_result = ''
        best_sim = -float('inf')
        
        for n, new_s in enumerate(all_test_senses[test_lemma]):
            
            new_super = wn.synset(new_s).lexname().replace('.', '_')
            super_vec = trainer._model.supersense_embeddings[new_super].view(1, -1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            cos_sim = cos(test_emb, super_vec)
            
            # find the best supersense prediction
            if cos_sim > best_sim:
                test_result = new_super
                best_sim = cos_sim
                
        correct_super = []
        for q, respon in enumerate(new_test_Y[test_idx]):
            if respon:
                correct_s = wn.synset(all_test_senses[test_lemma][q]).lexname().replace('.', '_')
                correct_super.append(correct_s) 
                
        # if the prediction is one of the supersense with annotator response being 1
        # regard as correct prediction
        if test_result in correct_super:
            unknown_correct_count += 1
            print('\ncorrect supersense prediction: ')
            print(test_sen)
            print('Word: {}'.format(test_sen[new_test_idx[test_idx]]))
            print('Predicted supersense: {}'.format(test_result))
        
    else:
            
        # if it is a known word
        known_test_size += 1
        
        # similarities to all its definition embeddings
        for k, sense in enumerate(all_senses[test_lemma]):
            definition_vec = trainer._model.definition_embeddings[test_lemma][:, k].view(1, -1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            cos_sim = cos(test_emb, definition_vec)
            all_similarity.append(cos_sim)

        test_result = all_similarity.index(max(all_similarity))
        if new_test_Y[test_idx][test_result] == 1:
            correct_count += 1
            if len(all_definitions[test_lemma][test_result]) > 1:
                print('\ncorrect sense prediction: ')
                print(test_sen)
                print('Word: {}'.format(test_sen[new_test_idx[test_idx]]))
                print('Predicted sense: {}'.format(all_definitions[test_lemma][test_result]))

print('\ntest size for known words: {}'.format(known_test_size))
print('accuracy for known words: {}'.format(correct_count / known_test_size))

print('test size for unknown words: {}'.format(unknown_test_size))
print('accuracy for unknown words: {}'.format(unknown_correct_count / unknown_test_size))



correct sense prediction: 
['Seth', 'provide', 'deep', 'tissue', 'massage', 'which', 'have', 'significantly', 'reduce', 'the', 'pain', 'in', 'my', 'neck', 'and', 'shoulder', 'and', 'add', 'flexibility', 'and', 'movement', 'back', 'to', 'the', 'area', '.']
Word: massage
Predicted sense: ['kneading', 'and', 'rubbing', 'parts', 'of', 'the', 'body', 'to', 'increase', 'circulation', 'and', 'promote', 'relaxation']

correct sense prediction: 
['name', 'something', 'you', 'find', 'at', 'a', 'carnival', 'that', 'come', 'on', 'a', 'stick', '?']
Word: stick
Predicted sense: ['a', 'small', 'thin', 'branch', 'of', 'a', 'tree']

correct sense prediction: 
['people', 'love', 'to', 'buy', 'these', 'cute', 'cuddly', 'little', 'animal', 'for', 'gift', 'and', 'collectable', '.']
Word: people
Predicted sense: ['(plural)', 'any', 'group', 'of', 'human', 'beings', '(men', 'or', 'women', 'or', 'children)', 'collectively']

correct sense prediction: 
['people', 'love', 'to', 'buy', 'these', 'cute', 'cuddly'


correct sense prediction: 
['room', 'be', 'clean', ',', 'plenty', 'of', 'thing', 'to', 'do', 'near', 'hotel', ',', 'and', 'safe', 'part', 'of', 'town', '.']
Word: room
Predicted sense: ['an', 'area', 'within', 'a', 'building', 'enclosed', 'by', 'walls', 'and', 'floor', 'and', 'ceiling']

correct sense prediction: 
['room', 'be', 'clean', ',', 'plenty', 'of', 'thing', 'to', 'do', 'near', 'hotel', ',', 'and', 'safe', 'part', 'of', 'town', '.']
Word: room
Predicted sense: ['an', 'area', 'within', 'a', 'building', 'enclosed', 'by', 'walls', 'and', 'floor', 'and', 'ceiling']

correct sense prediction: 
['so', 'i', 'live', 'in', 'Invercargill', 'New', 'Zealand', 'and', 'i', 'want', 'to', 'know', 'if', 'there', 'be', 'any', 'good', 'place', 'to', 'buy', 'a', 'ice', '-', 'cream', 'sundae', 'from', 'other', 'than', 'mc', 'donald', "'s", 'lol']
Word: place
Predicted sense: ['proper', 'or', 'appropriate', 'position', 'or', 'location']

correct sense prediction: 
['my', 'wife', 'and', 'I', 'would


correct sense prediction: 
['"', 'if', 'a', 'school', 'bus', 'carry', 'fifty', 'child', 'drive', 'off', 'a', 'cliff', ',', 'kill', 'everyone', 'involved', '...', 'that', 'would', 'be', 'a', 'tragedy', '"', '.']
Word: bus
Predicted sense: ['a', 'vehicle', 'carrying', 'many', 'passengers;', 'used', 'for', 'public', 'transport']

correct sense prediction: 
['he', 'make', 'some', 'good', 'observation', 'on', 'a', 'few', 'of', 'the', "pic'", '.']
Word: observation
Predicted sense: ['a', 'remark', 'expressing', 'careful', 'consideration']

correct sense prediction: 
['be', 'wonder', 'if', 'anyone', 'know', 'a', 'rough', 'estimate', 'of', 'how', 'much', 'it', 'cost', 'with', 'travel', 'and', 'training']
Word: travel
Predicted sense: ['the', 'act', 'of', 'going', 'from', 'one', 'place', 'to', 'another']

correct sense prediction: 
['be', 'wonder', 'if', 'anyone', 'know', 'a', 'rough', 'estimate', 'of', 'how', 'much', 'it', 'cost', 'with', 'travel', 'and', 'training']
Word: travel
Predicted se


correct sense prediction: 
['they', 'be', 'especially', 'rude', 'to', 'woman', '.']
Word: woman
Predicted sense: ['an', 'adult', 'female', 'person', '(as', 'opposed', 'to', 'a', 'man)']

correct sense prediction: 
['"', 'NASA', 'will', 'initiate', 'development', 'of', 'a', 'crew', 'launch', 'vehicle', 'derive', 'from', 'space', 'shuttle', 'solid', 'rocket', 'booster', 'with', 'a', 'new', 'upper', 'stage', 'for', 'human', 'spaceflight', ',', '"', 'say', 'the', 'letter', ',', 'sign', 'by', 'NASA', 'administrator', 'Mike', 'Griffin', 'and', 'the', 'Pentagon', "'s", 'top', 'space', 'official', ',', 'ex-astronaut', 'Ron', 'Sega', '...']
Word: letter
Predicted sense: ['a', 'written', 'message', 'addressed', 'to', 'a', 'person', 'or', 'organization']

correct sense prediction: 
['"', 'NASA', 'will', 'initiate', 'development', 'of', 'a', 'crew', 'launch', 'vehicle', 'derive', 'from', 'space', 'shuttle', 'solid', 'rocket', 'booster', 'with', 'a', 'new', 'upper', 'stage', 'for', 'human', 'space


correct supersense prediction: 
['even', 'the', 'least', 'discriminating', 'diner', 'would', 'know', 'not', 'to', 'eat', 'at', 'Sprecher', "'s", '.']
Word: diner
Predicted supersense: noun_artifact

correct sense prediction: 
['he', 'have', 'maintain', 'a', 'good', 'relationship', 'with', 'Mulva', '.']
Word: relationship
Predicted sense: ['a', 'state', 'of', 'connectedness', 'between', 'people', '(especially', 'an', 'emotional', 'connection)']

correct sense prediction: 
['he', 'have', 'maintain', 'a', 'good', 'relationship', 'with', 'Mulva', '.']
Word: relationship
Predicted sense: ['a', 'state', 'of', 'connectedness', 'between', 'people', '(especially', 'an', 'emotional', 'connection)']

correct sense prediction: 
['yeah', ',', 'i', 'be', 'think', 'somewhere', 'like', 'mcdonald', "'s", '.']
Word: somewhere
Predicted sense: ['an', 'indefinite', 'or', 'unknown', 'location']

correct sense prediction: 
['yeah', ',', 'i', 'be', 'think', 'somewhere', 'like', 'mcdonald', "'s", '.']
Word: 

...


correct sense prediction: 
['the', 'Pentagon', 'do', 'not', 'even', 'have', 'a', 'plan', 'for', 'deal', 'with', 'Afghanistan', 'or', 'al', '-', 'Qaeda', 'that', 'it', 'could', 'pull', 'off', 'the', 'shelf', ',', 'accord', 'to', 'Bob', 'Woodward', '.']
Word: plan
Predicted sense: ['a', 'series', 'of', 'steps', 'to', 'be', 'carried', 'out', 'or', 'goals', 'to', 'be', 'accomplished']

correct supersense prediction: 
['they', 'be', 'play', 'Captain', 'Ahab', 'to', 'Saddam', "'s", 'great', 'white', 'whale', '.']
Word: whale
Predicted supersense: noun_animal

correct supersense prediction: 
['imperial', 'hubris', 'make', 'the', 'case', 'that', 'lot', 'of', 'people', 'in', 'the', 'CIA', 'and', 'counter-terrorism', 'division', 'elsewhere', 'in', 'the', 'US', 'government', 'know', 'all', 'about', 'Bin', 'Laden', 'and', 'the', 'threat', 'he', 'pose', '.']
Word: hubris
Predicted supersense: noun_attribute

correct sense prediction: 
['imperial', 'hubris', 'make', 'the', 'case', 'that', 'lot', 'o


correct sense prediction: 
['look', 'at', 'a', 'map', 'and', 'you', 'try', 'to', 'figure', 'out', 'how', ',', 'in', 'fall', 'of', '1999', ',', 'you', 'could', 'possibly', 'pull', 'off', 'such', 'a', 'operation', 'without', 'pakistani', 'facility', '.']
Word: facility
Predicted sense: ['a', 'building', 'or', 'place', 'that', 'provides', 'a', 'particular', 'service', 'or', 'is', 'used', 'for', 'a', 'particular', 'industry']

correct sense prediction: 
['look', 'at', 'a', 'map', 'and', 'you', 'try', 'to', 'figure', 'out', 'how', ',', 'in', 'fall', 'of', '1999', ',', 'you', 'could', 'possibly', 'pull', 'off', 'such', 'a', 'operation', 'without', 'pakistani', 'facility', '.']
Word: fall
Predicted sense: ['the', 'season', 'when', 'the', 'leaves', 'fall', 'from', 'the', 'trees']

correct sense prediction: 
['but', 'for', 'those', 'of', 'you', 'tempt', 'in', 'that', 'direction', ',', 'please', 'look', 'up', 'Carter', "'s", 'Tabas', 'operation', '.']
Word: operation
Predicted sense: ['activity


correct supersense prediction: 
['not', 'sure', ',', 'but', 'i', 'assume', 'that', 'the', 'bluegrass', 'songbook', 'be', 'mine', '.']
Word: songbook
Predicted supersense: noun_communication

correct sense prediction: 
['we', 'have', 'this', 'report', '?']
Word: report
Predicted sense: ['a', 'written', 'document', 'describing', 'the', 'findings', 'of', 'some', 'individual', 'or', 'group']

correct sense prediction: 
['have', 'you', 'see', 'the', 'material', 'from', 'the', 'press', 'conference', 'that', 'he', 'launch', 'today', '?']
Word: material
Predicted sense: ['information', '(data', 'or', 'ideas', 'or', 'observations)', 'that', 'can', 'be', 'used', 'or', 'reworked', 'into', 'a', 'finished', 'form']

correct supersense prediction: 
['I', 'think', 'this', 'be', 'actually', 'a', 'good', 'thing', '--', 'make', 'we', 'proposal', 'look', 'like', 'a', 'much', 'more', 'preferable', 'alternative', 'by', 'comparison', ',', 'make', 'we', 'support', 'in', 'the', 'process', 'more', 'important'


correct sense prediction: 
['in', 'addition', ',', 'be', 'there', 'someone', 'who', 'could', 'consider', 'the', 'correlation', 'between', 'US', 'cpi', 'and', 'the', '$', '/', 'euro', 'exchange', 'rate', '.']
Word: correlation
Predicted sense: ['a', 'reciprocal', 'relation', 'between', 'two', 'or', 'more', 'things']

correct sense prediction: 
['in', 'addition', ',', 'be', 'there', 'someone', 'who', 'could', 'consider', 'the', 'correlation', 'between', 'US', 'cpi', 'and', 'the', '$', '/', 'euro', 'exchange', 'rate', '.']
Word: someone
Predicted sense: ['a', 'human', 'being']

correct sense prediction: 
['the', 'credit', 'guy', 'be', 'currently', 'assume', 'that', 'there', 'be', 'no', 'correlation', 'and', 'may', 'consequently', 'be', 'double', 'dip', 'the', 'credit', 'reserve', 'on', 'this', 'basis', 'too', '.']
Word: correlation
Predicted sense: ['a', 'reciprocal', 'relation', 'between', 'two', 'or', 'more', 'things']

correct sense prediction: 
['the', 'credit', 'guy', 'be', 'current


correct sense prediction: 
['they', 'have', 'always', 'do', 'a', 'great', 'job', 'at', 'a', 'reasonable', 'price', '.']
Word: job
Predicted sense: ['an', 'object', 'worked', 'on;', 'a', 'result', 'produced', 'by', 'working']

correct sense prediction: 
['"', 'thank', 'you', 'so', 'much', 'for', 'the', 'superior', 'job', 'well', 'do', '.']
Word: job
Predicted sense: ['an', 'object', 'worked', 'on;', 'a', 'result', 'produced', 'by', 'working']

correct sense prediction: 
['after', 'they', 'show', 'up', 'there', 'be', 'a', 'little', 'trouble', 'to', 'get', 'my', 'car', 'unlock', ',', 'it', 'take', 'quite', 'a', 'bit', 'of', 'time', 'but', 'the', 'job', 'be', 'well', 'do', '.']
Word: job
Predicted sense: ['an', 'object', 'worked', 'on;', 'a', 'result', 'produced', 'by', 'working']

test size for known words: 6440
accuracy for known words: 0.6209627329192546
test size for unknown words: 678
accuracy for unknown words: 0.6946902654867256


In [13]:
print('\ntest size for known words: {}'.format(known_test_size))
print('accuracy for known words: {}'.format(correct_count / known_test_size))

print('test size for unknown words: {}'.format(unknown_test_size))
print('accuracy for unknown words: {}'.format(unknown_correct_count / unknown_test_size))


test size for known words: 6440
accuracy for known words: 0.6209627329192546
test size for unknown words: 678
accuracy for unknown words: 0.6946902654867256


In [14]:
# problems
test_lemma = '____spring'
test_sen = ['the', 'spring', 'was', 'broken']
test_idx = 1
test_emb = trainer._model.forward(test_sen, test_idx).view(1, -1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
all_similarity = []

for k, sense in enumerate(all_senses[test_lemma]):
    definition_vec = trainer._model.definition_embeddings[test_lemma][:, k].view(1, -1).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    cos_sim = cos(test_emb, definition_vec)
    all_similarity.append(cos_sim)

test_result = all_similarity.index(max(all_similarity))
print(test_sen)
print('Word: {}'.format(test_sen[test_idx]))
print('Predicted sense: {}'.format(all_definitions[test_lemma][test_result]))

['the', 'spring', 'was', 'broken']
Word: spring
Predicted sense: ['the', 'season', 'of', 'growth']
